---
# 10.4 Divided differences and Newton's form
---

$%%% My LaTeX definitions
\DeclareMathOperator{\span}{span}
\newcommand{\Pbf}{\mathbf{P}}
$
The Lagrange polynomial basis can be thought of as

$$
\phi_j(x) = \prod_{\substack{i = 0 \\ i \neq j}}^n (x - x_i), 
\quad j = 0,1,\ldots,n.
$$

The **Newton polynomial** basis is defined in a very similar way:

$$
\phi_j(x) = \prod_{i = 0}^{j-1} (x - x_i), 
\quad j = 0,1,\ldots,n.
$$




Using the Newton basis, the matrix

$$
A = 
\begin{bmatrix}
\phi_0(x_0) & \phi_1(x_0) & \cdots & \phi_n(x_0)\\
\phi_0(x_1) & \phi_1(x_1) & \cdots & \phi_n(x_1)\\
\vdots & \vdots & \ddots & \vdots\\
\phi_0(x_n) & \phi_1(x_n) & \cdots & \phi_n(x_n)\\
\end{bmatrix}
$$

is **lower-triangular**. This means that we can solve $Ac = y$ using **forward-substitution** in $\mathcal{O}(n^2)$ flops.

---

## Example

The Newton polynomial basis for the data

$$
\begin{gather}
(x_0,y_0) = (2,1)\\
(x_1,y_1) = (6,2)\\
(x_2, y_2) = (4,3)\\
(x_3, y_3) = (8,2)\\
\end{gather}
$$

is the following set of polynomials

$$
\begin{align}
\phi_0(x) & = 1 \\
\phi_1(x) & = (x-2) \\
\phi_2(x) & = (x-2)(x-6) \\
\phi_3(x) & = (x-2)(x-6)(x-4) \\
\end{align}
$$

Using this basis, we find the interpolating polynomial

$$p(x) = c_0 \phi_0(x) + c_1 \phi_1(x) + c_2 \phi_2(x) + c_3 \phi_3(x)$$

by solving the linear system $Ac = y$, where

$$
A = 
\begin{bmatrix}
\phi_0(x_0) & \phi_1(x_0) & \phi_2(x_0) & \phi_3(x_0)\\
\phi_0(x_1) & \phi_1(x_1) & \phi_2(x_1) & \phi_3(x_1)\\
\phi_0(x_2) & \phi_1(x_2) & \phi_2(x_2) & \phi_3(x_2)\\
\phi_0(x_3) & \phi_1(x_3) & \phi_2(x_3) & \phi_3(x_3)\\
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0 & 0 \\
1 & 4 & 0 & 0 \\
1 & 2 & -4 & 0 \\
1 & 6 & 12 & 48 \\
\end{bmatrix}.
$$

Thus, $Ac = y$ is as follows.

$$
\begin{matrix}
c_0 & &       & &       & &       &=& 1 \\
c_0 &+& 4 c_1 & &       & &       &=& 2 \\
c_0 &+& 2 c_1 &-& 4c_2  & &       &=& 3 \\
c_0 &+& 6 c_1 &+& 12c_2 &+& 48c_3 &=& 2 \\
\end{matrix}
$$

We solve this system by forward-substitution:

$$
\begin{align}
c_0 &= 1 
\\
c_1 &= \frac{1}{4} (2 - c_0) = \frac{1}{4} (2 - 1) = \frac{1}{4} 
\\
c_2 &= -\frac{1}{4}(3 - c_0 - 2 c_1) = -\frac{1}{4}\left(3 - 1 - \frac{2}{4}\right) = -\frac38
\\
c_3 &= \frac{1}{48}(2 - c_0 - 6 c_1 - 12 c_2) = \frac{1}{48}\left(2 - 1 - \frac{6}{4}  + 12 \frac38\right) = \frac{1}{12}
\\
\end{align}
$$

In [ ]:
x = [2, 6, 4, 8]
y = [1, 2, 3, 2]

# Define A
A = Rational[1 0 0 0; 1 4 0 0; 1 2 -4 0; 1 6 12 48]

# Solve for the Newton polynomial coefficients
cnewt = A\y

Thus, the interpolating polynomial is 

$$
p(x) = 1 + \frac14(x-2) - \frac38(x-2)(x-6) + \frac{1}{12}(x-2)(x-6)(x-4)
$$

In [ ]:
########################################
function newtinterp(x::Vector, y::Vector)
    n = length(x)

    # Compute the matrix A corresponding to the Newton basis
    A = [prod(Float64[x[i] - x[k] for k=1:j]) for i=1:n, j=0:n-1]

    # Solve Ac = y
    c = A\y
end
########################################

x = [2, 6, 4, 8]
y = [1, 2, 3, 2]

newtinterp(x, y)

Let's check that we obtained the same polynomial as before.

In [ ]:
import SymPy

x = SymPy.symbols("x")

phinewt = [
    1,
    (x - 2),
    (x - 2)*(x - 6),
    (x - 2)*(x - 6)*(x - 4)
]

In [ ]:
cnewt

In [ ]:
using LinearAlgebra

pnewt = dot(cnewt, phinewt)

In [ ]:
pnewt = SymPy.expand(pnewt)

In [ ]:
x = [2, 6, 4, 8]
y = [1, 2, 3, 2]
n = length(x) - 1
A = Rational[xi^j for xi in x, j=0:n]
cmono = A\y

x = SymPy.symbols("x")
phimono = [x^j for j=0:3]
pmono = dot(cmono, phimono)

In [ ]:
pmono == pnewt

As expected, the interpolating polynomial obtained using the Newton basis matches the interpolating polynomial obtained from the monomial basis.

---

## Divided differences

Given the data points $\left\{(x_i,y_i)\right\}_{i=0}^n$, where $y_i = f(x_i)$, the Newton form of the interpolating polynomial can also be written in closed-form as:

$$p(x) = \sum_{j=0}^n c_j \phi_j(x) = \sum_{j=0}^n \left( f[x_0,\ldots,x_j] \prod_{i=0}^{j-1}(x-x_i) \right).$$

That is, the coefficient $c_j$ of $\phi_j(x)$ is the so-called **$j$th divided difference**:

$$c_j = f[x_0,\ldots,x_j].$$

Divided differences are defined recursively by

$$f[x_i] = f(x_i), \qquad \text{for } 1\leq i \leq n,$$

and

$$f[x_i,\ldots,x_j] = \frac{f[x_{i+1},\ldots,x_j] - f[x_i,\ldots,x_{j-1}]}{x_j - x_i}, \qquad \text{for } 1\leq i < j \leq n.$$

## Computing the divided differences

Divided differences can be nicely represented in a table as follows.

$$
\begin{array}{c||c|ccccc}
i & x_i & f[x_i] & f[x_{i-1},x_i] & f[x_{i-2},x_{i-1},x_i] & \cdots & f[x_0,\ldots,x_n] \\ \hline
0 & x_0 & f(x_0) \\
1 & x_1 & f(x_1) & f[x_0,x_1] \\
2 & x_2 & f(x_2) & f[x_1,x_2] & f[x_0,x_1,x_2] \\
\vdots & \vdots & \vdots & \vdots & \vdots & \ddots  \\
n & x_n & f(x_n) & f[x_{n-1},x_n] & f[x_{n-2},x_{n-1},x_n] & \cdots & f[x_0,\ldots,x_n] \\
\end{array}
$$

Note that the coefficients $c_j = f[x_0,\ldots,x_j]$ of the Newton polynomial appear along the diagonal of the table.

---

## Example

Let's complete the divided difference table for the data

$$
\begin{gather}
(x_0,y_0) = (2,1)\\
(x_1,y_1) = (6,2)\\
(x_2, y_2) = (4,3)\\
(x_3, y_3) = (8,2)\\
\end{gather}
$$

\begin{array}{c|c|cccc}
i & x_i & f[\cdot] & f[\cdot,\cdot] & f[\cdot,\cdot,\cdot] & f[\cdot,\cdot,\cdot,\cdot] \\
\hline
0 & 2 & 1 &  &  \\
1 & 6 & 2 & \frac{2-1}{6-2} &  \\
2 & 4 & 3 & \frac{3-2}{4-6} &  & \\
3 & 8 & 2 & \frac{2-3}{8-4} &  &  \\
\end{array}

\begin{array}{c|c|cccc}
i & x_i & f[\cdot] & f[\cdot,\cdot] & f[\cdot,\cdot,\cdot] & f[\cdot,\cdot,\cdot,\cdot] \\
\hline
0 & 2 & 1 &  &  \\
1 & 6 & 2 & \frac{1}{4} &  \\
2 & 4 & 3 & -\frac{1}{2} & \frac{-\frac12 - \frac14}{4-2} & \\
3 & 8 & 2 & -\frac{1}{4} & \frac{-\frac14 + \frac12}{8-6} &  \\
\end{array}

\begin{array}{c|c|cccc}
i & x_i & f[\cdot] & f[\cdot,\cdot] & f[\cdot,\cdot,\cdot] & f[\cdot,\cdot,\cdot,\cdot] \\
\hline
0 & 2 & 1 &  &  \\
1 & 6 & 2 & \frac{1}{4} &  \\
2 & 4 & 3 & -\frac{1}{2} & -\frac38 & \\
3 & 8 & 2 & -\frac{1}{4} & \frac18 & \frac{\frac18 + \frac38}{8-2} \\
\end{array}

\begin{array}{c|c|cccc}
i & x_i & f[\cdot] & f[\cdot,\cdot] & f[\cdot,\cdot,\cdot] & f[\cdot,\cdot,\cdot,\cdot] \\
\hline
0 & 2 & 1 &  &  \\
1 & 6 & 2 & \frac{1}{4} &  \\
2 & 4 & 3 & -\frac{1}{2} & -\frac38 & \\
3 & 8 & 2 & -\frac{1}{4} & \frac18 & \frac{1}{12} \\
\end{array}

Thus, the interpolating polynomial is 

$$
p(x) = 1 + \frac14(x-2) - \frac38(x-2)(x-6) + \frac{1}{12}(x-2)(x-6)(x-4).
$$

Note that the Newton form can be evaluated using a nested approach:

$$
p(x) = 1 + (x-2)\Bigg( \frac14 + (x-6)\bigg(-\frac38 + \frac{1}{12}(x-4)\bigg) \Bigg).
$$

---

## Computing the divided differences

In [ ]:
###############################
function divdif(x::Vector{T}, y::Vector{T}) where T

    n = length(x)
    Table = zeros(T, n, n)
    
    Table[:, 1] = y
    for k = 2:n
        for i=k:n
            Table[i, k] = Table[i, k-1] - Table[i-1, k-1]
            Table[i, k] /= x[i] - x[i-k+1]
        end
    end
    
    c = diag(Table)
    
    return c, Table
end
###############################

In [ ]:
x = [2, 6, 4, 8//1]
y = [1, 2, 3, 2//1]
c, T = divdif(x, y)
T

In [ ]:
###############################
function evalnewt(
        xspan::AbstractVector, 
        x::Vector, 
        c::Vector)
    n = length(x)
    
    p = c[n]*ones(length(xspan))
    for i = 1:length(p)
        for j = n-1:-1:1
            p[i] *= xspan[i] - x[j]
            p[i] += c[j]
        end
    end
    
    return p
end

evalnewt(xx::Real, x::Vector, c::Vector) = evalnewt([xx], x, c)[1]
###############################

In [ ]:
xx = [2, 6, 4, 8.]
yy = [1, 2, 3, 2.]
c, T = divdif(xx, yy)
c

In [ ]:
p(x) = evalnewt(x, xx, c)
p(4.0)

In [ ]:
using Plots, LaTeXStrings

a, b = 1, 9

plot(legend=:bottomright, xlabel=L"x", ylabel=L"y")
plot!(p, a, b, label=L"y = p(x)")
scatter!(xx, yy, label=:none, c=1)

---

> ## Theorem: (Divided Differences and Newton's Form)
>
>Given the data points $\left\{(x_i,y_i)\right\}_{i=0}^n$, where $y_i = f(x_i)$, the Newton form of the interpolating polynomial can be written as:
>
>$$p(x) = \sum_{j=0}^n \left( f[x_0,\ldots,x_j] \prod_{i=0}^{j-1}(x-x_i) \right),$$
>
>where the divided differences are defined recursively by
>
>$$f[x_i] = f(x_i), \qquad \text{for } 1\leq i \leq n,$$
>
>and
>
>$$f[x_i,\ldots,x_j] = \frac{f[x_{i+1},\ldots,x_j] - f[x_i,\ldots,x_{j-1}]}{x_j - x_i}, \qquad \text{for } 1\leq i < j \leq n.$$

## Proof:

We will prove this fact by induction on $n$. When $n=1$, we are interpolating the data points $(x_0, f(x_0))$ and $(x_1, f(x_1))$ using the basis

$$
\begin{align}
\phi_0(x) & = 1, \\
\phi_1(x) & = (x-x_0). \\
\end{align}
$$

Thus we want to find $c_0$ and $c_1$ such that

$$p(x) = c_0 + c_1 (x-x_0)$$

satisfies $p(x_0) = f(x_0)$ and $p(x_1) = f(x_1)$. This gives us the following linear system.

$$
\begin{matrix}
c_0 & &                 &=& f(x_0) \\
c_0 &+& (x_1 - x_0) c_1 &=& f(x_1) \\
\end{matrix}
$$

Thus,

$$c_0 = f(x_0) = f[x_0], \qquad \text{and} \qquad c_1 = \frac{f(x_1)-f(x_0)}{x_1-x_0} = \frac{f[x_1]-f[x_0]}{x_1-x_0} = f[x_0,x_1],$$

as required.

Now we suppose the result is true for $n = k$, and we will prove that this implies the result is true for $n = k+1$. 

Let $p(x)$ be the unique polynomial of degree at most $k+1$ that interpolates the $k+2$ points $(x_0, f(x_0)), \ldots, (x_{k+1}, f(x_{k+1}))$. 

Let $q_0(x)$ be the unique polynomial of degree at most $k$ that interpolates the first $k+1$ points $(x_0, f(x_0)), \ldots, (x_{k}, f(x_{k}))$.

Let $q_1(x)$ be the unique polynomial of degree at most $k$ that interpolates the last $k+1$ points $(x_1, f(x_1)), \ldots, (x_{k+1}, f(x_{k+1}))$. 

By the induction assumption, we have

$$
q_0(x) = \sum_{j=0}^{k} \left( f[x_0,\ldots,x_j] \prod_{i=0}^{j-1}(x-x_i) \right)
$$
and
$$
q_1(x) = \sum_{j=1}^{k+1} \left( f[x_1,\ldots,x_j] \prod_{i=1}^{j-1}(x-x_i) \right).
$$

To obtain $p(x)$ we just need to add to $q_0(x)$ a multiple of the basis polynomial $\prod_{i=0}^k (x-x_i)$:

$$p(x) = q_0(x) + c_{k+1} \prod_{i=0}^k (x-x_i).$$

If $c_{k+1}$ satisfies

$$f(x_{k+1}) = q_0(x_{k+1}) + c_{k+1} \prod_{i=0}^k (x_{k+1}-x_i),$$

then $p(x)$ will interpolate $(x_0, f(x_0)), \ldots, (x_{k+1}, f(x_{k+1}))$.

In a similar way, we can add to $q_1(x)$ a multiple of the basis polynomial $\prod_{i=1}^{k+1} (x-x_i)$ to obtain $p(x)$. Thus,

$$p(x) = q_1(x) + c_{k+1}' \prod_{i=1}^{k+1} (x-x_i),$$

where $c_{k+1}'$ satisfies

$$f(x_0) = q_1(x_0) + c_{k+1}' \prod_{i=1}^{k+1} (x_0-x_i).$$


Now we have that 

$$q_0(x) + c_{k+1} \prod_{i=0}^k (x-x_i) = q_1(x) + c_{k+1}' \prod_{i=1}^{k+1} (x-x_i).$$

Thus

$$
\frac{d^{k+1}}{dx^{k+1}}\left(q_0(x) + c_{k+1} \prod_{i=0}^k (x-x_i)\right) 
= 
\frac{d^{k+1}}{dx^{k+1}}\left(q_1(x) + c_{k+1}' \prod_{i=1}^{k+1} (x-x_i)\right).
$$

Since $q_0(x)$ and $q_1(x)$ have degree at most $k$, and 

$$
\prod_{i=0}^k (x-x_i) = x^{k+1} + \cdots
\qquad
\text{and}
\qquad
\prod_{i=1}^{k+1} (x-x_i) = x^{k+1} + \cdots,
$$

we have 

$$0 + c_{k+1}(k+1)! = 0 + c_{k+1}'(k+1)!,$$

which implies $c_{k+1} = c_{k+1}'$.



Thus we have

$$
q_0(x) + c_{k+1} \prod_{i=0}^k (x-x_i) 
= 
q_1(x) + c_{k+1} \prod_{i=1}^{k+1} (x-x_i).
$$

Now consider 

$$
\frac{d^k}{dx^k}\left(q_0(x) + c_{k+1} \prod_{i=0}^k (x-x_i)\right)
= 
\frac{d^k}{dx^k}\left(q_1(x) + c_{k+1} \prod_{i=1}^{k+1} (x-x_i)\right).
$$

Note that

$$
q_0(x) = f[x_0,\ldots,x_k] x^k + \cdots
\quad \text{and} \quad
q_1(x) = f[x_1,\ldots,x_{k+1}] x^k + \cdots.
$$

Also,

$$
\prod_{i=0}^k (x-x_i) = x^{k+1} - x^{k}\sum_{i=0}^k x_i + \cdots
\quad \text{and} \quad
\prod_{i=1}^{k+1} (x-x_i) = x^{k+1} - x^{k}\sum_{i=1}^{k+1} x_i + \cdots.
$$

Thus,

$$
k! f[x_0,\ldots,x_k] + c_{k+1} \left((k+1)!x - k! \sum_{i=0}^k x_i\right)
$$
$$
=
k! f[x_1,\ldots,x_{k+1}] + c_{k+1} \left((k+1)!x - k! \sum_{i=1}^{k+1} x_i\right).
$$

Dividing by $k!$, and rearranging, we have

$$
c_{k+1} \left((k+1)x - \sum_{i=0}^k x_i\right) - c_{k+1} \left((k+1)x - \sum_{i=1}^{k+1} x_i\right)
=
f[x_1,\ldots,x_{k+1}] - f[x_0,\ldots,x_k].
$$

Canceling common terms, we have

$$
c_{k+1} \left(\sum_{i=1}^{k+1} x_i - \sum_{i=0}^k x_i\right)
=
f[x_1,\ldots,x_{k+1}] - f[x_0,\ldots,x_k],
$$

which becomes

$$
c_{k+1} \left(x_{k+1} - x_0\right)
=
f[x_1,\ldots,x_{k+1}] - f[x_0,\ldots,x_k].
$$

Finally,

$$
c_{k+1}
=
\frac{f[x_1,\ldots,x_{k+1}] - f[x_0,\ldots,x_k]}{x_{k+1} - x_0} = f[x_0,\ldots,x_{k+1}].
$$

Therefore,

$$p(x) = q_0(x) + c_{k+1} \prod_{i=0}^k (x-x_i) = \sum_{j=0}^{k+1} \left( f[x_0,\ldots,x_j] \prod_{i=0}^{j-1}(x-x_i) \right),$$

as required. $\blacksquare$


---

## Pros and cons of Newton interpolation

Pros:

1. **Constructing $p(x)$ is fast**: roughly $\frac32n^2$ flops to compute the coefficients
2. **Evaluating $p(x)$ is fast**: about $3n$ flops to compute $p(x)$ compared to $2n$ flops using Horner's Rule
3. **Adding a new interpolation point is fast**: table of divided differences can be updated in $\mathcal{O}(n)$ flops
4. **Can be extended to also interpolate derivative values** (Section 10.7)

Cons:

4. **Cannot easily change the function**: the entire table of divided differences changes when using a different function $f$


---